In [1]:
import os
import sys

module_path = os.path.abspath(os.path.join('../..'))

if module_path not in sys.path:
    sys.path.append(module_path)


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


from math import sqrt
from tensorflow import keras
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import *


from preprocess import water_process
# from model.cnn import cnn_4layer, mlp_layer
from model.mlp import mlp_layer

2024-01-20 14:38:36.601079: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [4]:
dataset = pd.read_csv('../../data/Merged-update_hourly.csv', index_col=0)
dataset.fillna(0, inplace=True)
data = dataset
data

,WS_S1,WS_S4,FLOW_S25A,GATE_S25A,HWS_S25A,TWS_S25A,FLOW_S25B,GATE_S25B,GATE_S25B2,HWS_S25B,TWS_S25B,PUMP_S25B,FLOW_S26,GATE_S26_1,GATE_S26_2,HWS_S26,TWS_S26,PUMP_S26,MEAN_RAIN
Time,,,,,,,,,,,,,,,,,,,
1/1/10 00:00,0.33,0.34,0.0,0.0,2.17,0.49,0.0,0.0,0.0,2.84,0.58,0.0,0.00,0.0,0.00,2.78,0.505,0.0,0.0
1/1/10 01:00,-0.29,-0.28,0.0,0.0,2.17,0.02,0.0,0.0,0.0,2.84,-0.07,0.0,0.00,0.0,0.00,2.78,-0.110,0.0,0.0
1/1/10 02:00,-0.81,-0.73,0.0,0.0,2.17,-0.49,0.0,0.0,0.0,2.84,-0.62,0.0,0.00,0.0,0.00,2.79,-0.650,0.0,0.0
1/1/10 03:00,-0.98,-0.85,0.0,0.0,2.15,-0.75,0.0,0.0,0.0,2.83,-0.82,0.0,0.00,0.0,0.00,2.79,-0.850,0.0,0.0
1/1/10 04:00,-0.69,-0.49,0.0,0.0,2.13,-0.55,0.0,0.0,0.0,2.84,-0.53,0.0,0.00,0.0,0.00,2.79,-0.550,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12/31/20 19:00,1.24,1.19,0.0,0.0,1.92,1.25,0.0,0.0,0.0,2.86,1.31,0.0,456.34,0.0,2.61,2.49,1.290,0.0,0.0
12/31/20 20:00,1.62,1.61,0.0,0.0,1.92,1.64,0.0,0.0,0.0,2.88,1.68,0.0,380.19,0.0,2.62,2.48,1.660,0.0,0.0
12/31/20 21:00,1.98,1.90,0.0,0.0,1.93,2.00,0.0,0.0,0.0,2.89,2.04,0.0,287.12,0.0,2.62,2.54,2.040,0.0,0.0


In [5]:
np.percentile(data['MEAN_RAIN'], 95)

0.0053125

### Preprocess

In [4]:
# ====== preprocessing parameters ======
n_hours = 72
k = 24 
masked_value = 1e-10
split_1 = 0.7
split_2 = 0.85

In [4]:
train_X_mask, val_X_mask, test_X_mask, \
train_ws_y, val_ws_y, test_ws_y, \
scaler, ws_scaler = water_process(n_hours, k, masked_value, split_1, split_2)

Index(['MEAN_RAIN', 'WS_S4', 'GATE_S25A', 'GATE_S25B', 'GATE_S25B2',
       'GATE_S26_1', 'GATE_S26_2', 'PUMP_S25B', 'PUMP_S26', 'HWS_S25A',
       'HWS_S25B', 'HWS_S26', 'WS_S1', 'TWS_S25A', 'TWS_S25B', 'TWS_S26'],
      dtype='object')


In [5]:
print(train_X_mask.shape, val_X_mask.shape, test_X_mask.shape, train_ws_y.shape, val_ws_y.shape, test_ws_y.shape)

(67435, 96, 16) (14451, 96, 16) (14451, 96, 16) (67435, 96) (14451, 96) (14451, 96)


In [5]:
data = pd.read_csv("../../data/Merged-update_hourly.csv", index_col=0)
data.fillna(0, inplace=True)
data

,WS_S1,WS_S4,FLOW_S25A,GATE_S25A,HWS_S25A,TWS_S25A,FLOW_S25B,GATE_S25B,GATE_S25B2,HWS_S25B,TWS_S25B,PUMP_S25B,FLOW_S26,GATE_S26_1,GATE_S26_2,HWS_S26,TWS_S26,PUMP_S26,MEAN_RAIN
Time,,,,,,,,,,,,,,,,,,,
1/1/10 00:00,0.33,0.34,0.0,0.0,2.17,0.49,0.0,0.0,0.0,2.84,0.58,0.0,0.00,0.0,0.00,2.78,0.505,0.0,0.0
1/1/10 01:00,-0.29,-0.28,0.0,0.0,2.17,0.02,0.0,0.0,0.0,2.84,-0.07,0.0,0.00,0.0,0.00,2.78,-0.110,0.0,0.0
1/1/10 02:00,-0.81,-0.73,0.0,0.0,2.17,-0.49,0.0,0.0,0.0,2.84,-0.62,0.0,0.00,0.0,0.00,2.79,-0.650,0.0,0.0
1/1/10 03:00,-0.98,-0.85,0.0,0.0,2.15,-0.75,0.0,0.0,0.0,2.83,-0.82,0.0,0.00,0.0,0.00,2.79,-0.850,0.0,0.0
1/1/10 04:00,-0.69,-0.49,0.0,0.0,2.13,-0.55,0.0,0.0,0.0,2.84,-0.53,0.0,0.00,0.0,0.00,2.79,-0.550,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12/31/20 19:00,1.24,1.19,0.0,0.0,1.92,1.25,0.0,0.0,0.0,2.86,1.31,0.0,456.34,0.0,2.61,2.49,1.290,0.0,0.0
12/31/20 20:00,1.62,1.61,0.0,0.0,1.92,1.64,0.0,0.0,0.0,2.88,1.68,0.0,380.19,0.0,2.62,2.48,1.660,0.0,0.0
12/31/20 21:00,1.98,1.90,0.0,0.0,1.93,2.00,0.0,0.0,0.0,2.89,2.04,0.0,287.12,0.0,2.62,2.54,2.040,0.0,0.0


### Model

In [6]:
# ===== model parameters ======
mlp_unit1 = 128
mlp_unit2 = 128
mlp_unit3 = 64
mlp_unit4 = 64
mlp_unit5 = 32
mlp_unit6 = 32
mlp_unit7 = 16
mlp_unit8 = 16
dropout = 0.0  # 0.1
kernel_size = 2
pool_size = 2
learning_rate = 1e-4
decay_steps = 10000
decay_rate = 0.95
PATIENCE = 100
EPOCHS = 1000
BATCH = 512
opt_num = k*4
input_shape = train_X_mask.shape[1:]

In [9]:
model = mlp_layer(input_shape=input_shape,
                   mlp_unit1=mlp_unit1,
                   mlp_unit2=mlp_unit2,
                   mlp_unit3=mlp_unit3,
                   mlp_unit4=mlp_unit4,
                   mlp_unit5=mlp_unit5,
                   mlp_unit6=mlp_unit6,
                   mlp_unit7=mlp_unit7,
                   mlp_unit8=mlp_unit8,
                   dropout=dropout,
                   masked_value=-1,
                   opt_num=opt_num
                  )
# model.summary()

In [8]:
lr_schedule = keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=learning_rate, 
                                                          decay_steps=decay_steps,
                                                          decay_rate=decay_rate)

model.compile(optimizer=Adam(learning_rate=lr_schedule),
              loss='mse',
              metrics=['mae']
             )


es = EarlyStopping(monitor='val_mae', mode='min', verbose=2, patience=PATIENCE)
mc = ModelCheckpoint('../../saved_models/water_all_95.h5', 
                     monitor='val_mae', 
                     mode='min', 
                     verbose=2, 
                     save_best_only=True
                    )


model.fit([train_X_mask], train_ws_y,
          validation_data=([val_X_mask], val_ws_y),
          epochs=EPOCHS,
          batch_size=BATCH,
          verbose=2,
          shuffle=True,
          callbacks=[es, mc]
         )

2024-01-13 12:49:27.706171: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2024-01-13 12:49:27.706950: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 1700105000 Hz


Epoch 1/1000


2024-01-13 12:49:29.177880: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10


132/132 - 5s - loss: 0.0405 - mae: 0.1557 - val_loss: 0.0221 - val_mae: 0.1250

Epoch 00001: val_mae improved from inf to 0.12496, saving model to ../../saved_models/water_all.h5
Epoch 2/1000
132/132 - 2s - loss: 0.0061 - mae: 0.0609 - val_loss: 0.0049 - val_mae: 0.0557

Epoch 00002: val_mae improved from 0.12496 to 0.05572, saving model to ../../saved_models/water_all.h5
Epoch 3/1000
132/132 - 2s - loss: 0.0016 - mae: 0.0308 - val_loss: 0.0037 - val_mae: 0.0485

Epoch 00003: val_mae improved from 0.05572 to 0.04851, saving model to ../../saved_models/water_all.h5
Epoch 4/1000
132/132 - 2s - loss: 0.0014 - mae: 0.0285 - val_loss: 0.0032 - val_mae: 0.0449

Epoch 00004: val_mae improved from 0.04851 to 0.04495, saving model to ../../saved_models/water_all.h5
Epoch 5/1000
132/132 - 2s - loss: 0.0012 - mae: 0.0268 - val_loss: 0.0027 - val_mae: 0.0419

Epoch 00005: val_mae improved from 0.04495 to 0.04188, saving model to ../../saved_models/water_all.h5
Epoch 6/1000
132/132 - 2s - loss: 0.0

132/132 - 2s - loss: 1.5352e-04 - mae: 0.0076 - val_loss: 0.0011 - val_mae: 0.0270

Epoch 00049: val_mae did not improve from 0.02682
Epoch 50/1000
132/132 - 2s - loss: 1.5276e-04 - mae: 0.0076 - val_loss: 0.0011 - val_mae: 0.0269

Epoch 00050: val_mae did not improve from 0.02682
Epoch 51/1000
132/132 - 2s - loss: 1.5081e-04 - mae: 0.0076 - val_loss: 0.0011 - val_mae: 0.0269

Epoch 00051: val_mae did not improve from 0.02682
Epoch 52/1000
132/132 - 2s - loss: 1.4944e-04 - mae: 0.0075 - val_loss: 0.0011 - val_mae: 0.0275

Epoch 00052: val_mae did not improve from 0.02682
Epoch 53/1000
132/132 - 2s - loss: 1.4846e-04 - mae: 0.0075 - val_loss: 0.0011 - val_mae: 0.0266

Epoch 00053: val_mae improved from 0.02682 to 0.02660, saving model to ../../saved_models/water_all.h5
Epoch 54/1000
132/132 - 2s - loss: 1.4654e-04 - mae: 0.0074 - val_loss: 0.0010 - val_mae: 0.0262

Epoch 00054: val_mae improved from 0.02660 to 0.02621, saving model to ../../saved_models/water_all.h5
Epoch 55/1000
132/13

...


Epoch 00786: val_mae did not improve from 0.01700
Epoch 787/1000
132/132 - 2s - loss: 5.7930e-05 - mae: 0.0044 - val_loss: 5.7748e-04 - val_mae: 0.0173

Epoch 00787: val_mae did not improve from 0.01700
Epoch 788/1000
132/132 - 2s - loss: 5.7649e-05 - mae: 0.0044 - val_loss: 5.6540e-04 - val_mae: 0.0171

Epoch 00788: val_mae did not improve from 0.01700
Epoch 789/1000
132/132 - 2s - loss: 5.7783e-05 - mae: 0.0044 - val_loss: 5.9080e-04 - val_mae: 0.0172

Epoch 00789: val_mae did not improve from 0.01700
Epoch 790/1000
132/132 - 2s - loss: 5.7768e-05 - mae: 0.0044 - val_loss: 5.8470e-04 - val_mae: 0.0174

Epoch 00790: val_mae did not improve from 0.01700
Epoch 791/1000
132/132 - 2s - loss: 5.7971e-05 - mae: 0.0045 - val_loss: 5.8762e-04 - val_mae: 0.0173

Epoch 00791: val_mae did not improve from 0.01700
Epoch 792/1000
132/132 - 2s - loss: 5.7563e-05 - mae: 0.0044 - val_loss: 5.7828e-04 - val_mae: 0.0172

Epoch 00792: val_mae did not improve from 0.01700
Epoch 793/1000
132/132 - 2s - l

132/132 - 2s - loss: 5.6968e-05 - mae: 0.0044 - val_loss: 5.6982e-04 - val_mae: 0.0172

Epoch 00840: val_mae did not improve from 0.01693
Epoch 841/1000
132/132 - 2s - loss: 5.6896e-05 - mae: 0.0044 - val_loss: 5.7095e-04 - val_mae: 0.0171

Epoch 00841: val_mae did not improve from 0.01693
Epoch 842/1000
132/132 - 2s - loss: 5.7069e-05 - mae: 0.0044 - val_loss: 5.4899e-04 - val_mae: 0.0169

Epoch 00842: val_mae improved from 0.01693 to 0.01690, saving model to ../../saved_models/water_all.h5
Epoch 843/1000
132/132 - 2s - loss: 5.7608e-05 - mae: 0.0044 - val_loss: 5.8404e-04 - val_mae: 0.0173

Epoch 00843: val_mae did not improve from 0.01690
Epoch 844/1000
132/132 - 2s - loss: 5.7188e-05 - mae: 0.0044 - val_loss: 5.8353e-04 - val_mae: 0.0173

Epoch 00844: val_mae did not improve from 0.01690
Epoch 845/1000
132/132 - 2s - loss: 5.6907e-05 - mae: 0.0044 - val_loss: 5.7055e-04 - val_mae: 0.0171

Epoch 00845: val_mae did not improve from 0.01690
Epoch 846/1000
132/132 - 2s - loss: 5.6895e-

132/132 - 2s - loss: 5.6318e-05 - mae: 0.0044 - val_loss: 5.7741e-04 - val_mae: 0.0172

Epoch 00893: val_mae did not improve from 0.01688
Epoch 894/1000
132/132 - 2s - loss: 5.6587e-05 - mae: 0.0044 - val_loss: 5.6837e-04 - val_mae: 0.0171

Epoch 00894: val_mae did not improve from 0.01688
Epoch 895/1000
132/132 - 2s - loss: 5.6455e-05 - mae: 0.0044 - val_loss: 5.7683e-04 - val_mae: 0.0172

Epoch 00895: val_mae did not improve from 0.01688
Epoch 896/1000
132/132 - 2s - loss: 5.6371e-05 - mae: 0.0044 - val_loss: 5.7163e-04 - val_mae: 0.0171

Epoch 00896: val_mae did not improve from 0.01688
Epoch 897/1000
132/132 - 2s - loss: 5.6213e-05 - mae: 0.0044 - val_loss: 5.8841e-04 - val_mae: 0.0172

Epoch 00897: val_mae did not improve from 0.01688
Epoch 898/1000
132/132 - 2s - loss: 5.6584e-05 - mae: 0.0044 - val_loss: 5.8071e-04 - val_mae: 0.0171

Epoch 00898: val_mae did not improve from 0.01688
Epoch 899/1000
132/132 - 2s - loss: 5.6174e-05 - mae: 0.0044 - val_loss: 5.6957e-04 - val_mae: 0.


Epoch 00946: val_mae did not improve from 0.01688
Epoch 947/1000
132/132 - 2s - loss: 5.5952e-05 - mae: 0.0044 - val_loss: 5.7670e-04 - val_mae: 0.0172

Epoch 00947: val_mae did not improve from 0.01688
Epoch 948/1000
132/132 - 2s - loss: 5.5804e-05 - mae: 0.0044 - val_loss: 5.6902e-04 - val_mae: 0.0171

Epoch 00948: val_mae did not improve from 0.01688
Epoch 949/1000
132/132 - 2s - loss: 5.5679e-05 - mae: 0.0044 - val_loss: 5.6991e-04 - val_mae: 0.0173

Epoch 00949: val_mae did not improve from 0.01688
Epoch 950/1000
132/132 - 2s - loss: 5.5750e-05 - mae: 0.0044 - val_loss: 5.7038e-04 - val_mae: 0.0171

Epoch 00950: val_mae did not improve from 0.01688
Epoch 951/1000
132/132 - 2s - loss: 5.5973e-05 - mae: 0.0044 - val_loss: 5.8956e-04 - val_mae: 0.0173

Epoch 00951: val_mae did not improve from 0.01688
Epoch 952/1000
132/132 - 2s - loss: 5.6047e-05 - mae: 0.0044 - val_loss: 5.9108e-04 - val_mae: 0.0173

Epoch 00952: val_mae did not improve from 0.01688
Epoch 953/1000
132/132 - 2s - l

132/132 - 2s - loss: 5.5099e-05 - mae: 0.0043 - val_loss: 5.4788e-04 - val_mae: 0.0169

Epoch 01000: val_mae did not improve from 0.01682
